## I - Invoking APIs and inserting semi-structured data into MongDB
#### 1) Importing libraries

In [1]:
from sodapy import Socrata
from pymongo import MongoClient
import sys
print("Imported all the required libraries")

Imported all the required libraries


#### 2) Fetch Data from API

In [2]:
# Creating authentication client:
try:
    client = Socrata("data.medicare.gov", "TeKcMRRNIEgHqcTM3vybaqF5V",username="x19146281@student.ncirl.ie",password="Hornet@123")
    # fetching the results -  Hospital Readmissions Reduction Program (hrrp),
    # Hospital-Acquired Condition Reduction Program (hacrp), Hospital Value-Based Purchasing (hvbp)
    results_hrrp = client.get("9n3s-kdb3", limit=20000)
    results_hacrp = client.get("yq43-i98g", limit=20000)
    results_hvbp = client.get("pudb-wetr", limit=20000) 
except:
    print("com.socrata.exception- {}".format(sys.exc_info()[1]))
else:
    print("All JSONs fetched successfully via APIs")

All JSONs fetched successfully via APIs


#### 3) Initialize the MongoDB (DAP_ProjectDB) hosted on server "cyclades.okeanos-global.grnet.gr"

In [3]:
try:
    db_cl = MongoClient('83.212.82.56', 27017)
    db = db_cl['DAP_ProjectDB']
except:
    print("Failed to initialise- {}".format(sys.exc_info()[1]))
else:
    print("MongoDB instance initialized!")


MongoDB instance initialized!


#### 4) Insert Data to MongoDB

In [4]:
collection_hrrp = db['hrrp']
collection_hacrp = db['hacrp']
collection_hvbp = db['hvbp']
try:
    print("Removing collections if already present")
    if 'hrrp' in db.list_collection_names() :
        collection_hrrp.drop()
        print("Dropped collection hrrp")
    if 'hacrp' in db.list_collection_names() :
        collection_hacrp.drop()
        print("Dropped collection hacrp")
    if 'hvbp' in db.list_collection_names() :
        collection_hvbp.drop()
        print("Dropped collection hvbp")
    collection_hrrp.insert_many(results_hrrp)
    print("Insertion into collection hrrp done")
    collection_hacrp.insert_many(results_hacrp)
    print("Insertion into collection hacrp done")
    collection_hvbp.insert_many(results_hvbp)
    print("Insertion into collection hvbp done")
except:
    print("Failed to connect to MongoDB- {}".format(sys.exc_info()[1]))


Removing collections if already present
Insertion into collection hrrp done
Insertion into collection hacrp done
Insertion into collection hvbp done


#### 5) Checking the size of collection and structure of documents

In [5]:
print("Size of collection/ number of documents : ")
print(f" hrrp : {db['hrrp'].count_documents({})}")
print(f" hacrp : {db['hacrp'].count_documents({})}")
print(f" hvbp : {db['hvbp'].count_documents({})}")

print("\nStructure of documents : ")
print(f" hrrp : {list(db['hrrp'].find_one().keys())}\n")
print(f" hacrp : {list(db['hacrp'].find_one().keys())}\n")
print(f" hvbp : {list(db['hvbp'].find_one().keys())}\n")


Size of collection/ number of documents : 
 hrrp : 19344
 hacrp : 3224
 hvbp : 2731

Structure of documents : 
 hrrp : ['_id', 'hospital_name', 'provider_id', 'state', 'measure_name', 'number_of_discharges', 'excess_readmission_ratio', 'predicted_readmission_rate', 'expected_readmission_rate', 'number_of_readmissions', 'start_date', 'end_date']

 hacrp : ['_id', 'provider_id', 'hospital_name', 'state', 'fiscal_year', 'psi_90__start_date', 'psi_90_end_date', 'psi_90_w_z_score', 'clabsi_w_z_score', 'cauti_w_z_score', 'ssi_w_z_score', 'mrsa_w_z_score', 'cdi_w_z_score', 'hai_measures_start_date', 'hai_measures_end_date', 'total_hac_score', 'payment_reduction']

 hvbp : ['_id', 'provider_number', 'hospital_name', 'address', 'city', 'state', 'zip_code', 'county_name', 'mort_30_ami_achievement_threshold', 'mort_30_ami_benchmark', 'mort_30_ami_baseline_rate', 'mort_30_ami_performance_rate', 'mort_30_ami_achievement_points', 'mort_30_ami_improvement_points', 'mort_30_ami_measure_score', 'mort_3